In [4]:
# 4. 신경망 모델 구성하기
#신경망은 데이터에 대한 연산을 수행하는 layer/module로 구성됨.
#torch.nn 신경망 구성 요소 제공

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [9]:
# 4-1. 학습을 위한 장치 얻기 (GPU 모델로) (CUDA 10.2 버전 지원 x)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print('Using {} device'.format(device))

device = torch.device('cpu')

In [10]:
# 4-2. 클래스 정의하기
#신경망 모델을 nn.Module의 하위클래스로 정의, __init__ 신경망 계층들 초기화
#nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산 구현
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
    
#NeuralNetwork의 인스턴스 생성 후 device로 이동한 뒤, structure 출력 (10-차원 텐서 반환됨)
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [12]:
#원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률 얻음
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [13]:
# 4-3. 모델 계층(Layer)
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [14]:
#nn.Flatten: 계층을 초기화하여 28x28의 2D이미지를 784픽셀 값을 갖는 연속된 배열로 변환
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [15]:
#nn.Linear: 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 linear transformation을 적용하는 모듈
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [16]:
#nn.ReLU: 비선형 활성화(activation), 모델의 입력과 출력 사이 복잡한 관계(mapping)을 만듦.
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1383,  0.0280, -0.8452,  0.5328, -0.3093, -0.3265,  0.2363,  0.2962,
          0.0051, -0.1414, -0.0347,  0.0662,  0.1580,  0.0046, -0.1316,  0.7648,
          0.3470, -0.5088, -0.2353, -0.2216],
        [ 0.3016, -0.3619, -0.5546,  0.6903, -0.3972, -0.1583,  0.2376, -0.1282,
         -0.1458,  0.0034, -0.0890,  0.1028,  0.3383,  0.2761, -0.4433,  0.7410,
         -0.0225, -0.0366, -0.3086, -0.0139],
        [ 0.2303, -0.1368, -0.6533,  0.5481, -0.1284, -0.1842,  0.0580, -0.0150,
         -0.4142, -0.1455,  0.1574,  0.1040, -0.0828, -0.1969, -0.6088,  0.7342,
          0.3211, -0.1713, -0.3394,  0.0248]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1383, 0.0280, 0.0000, 0.5328, 0.0000, 0.0000, 0.2363, 0.2962, 0.0051,
         0.0000, 0.0000, 0.0662, 0.1580, 0.0046, 0.0000, 0.7648, 0.3470, 0.0000,
         0.0000, 0.0000],
        [0.3016, 0.0000, 0.0000, 0.6903, 0.0000, 0.0000, 0.2376, 0.0000, 0.0000,
         0.0034, 0.0000, 0.1028, 0.3383, 0.2761, 0.000

In [17]:
#nn.Sequential: 순서를 갖는 module의 컨테이너 (순차 컨테이너)
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [18]:
#nn.Softmax: 신경망의 마지막 선형 계층, [-infty, infty]범위의 원시 값(raw value)인 logits 반환
#logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정(scale)됨.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [21]:
# 4-4. 모델 매개변수
#신경망 내부의 많은 계층들은 매개변수화 됨 (학습 중에 최적화되는 weight와 bias와 연관지어진다.)
#nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 track되며, parameters()및 named_parameters() 메소드로 접근가능
print("Model structure: ", model, "\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0248, -0.0091, -0.0328,  ...,  0.0258,  0.0322,  0.0222],
        [-0.0213,  0.0149,  0.0345,  ..., -0.0142, -0.0282, -0.0152]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0228,  0.0355], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0265,  0.0341, -0.0413,  ..., -0.0266, -0.0378,  0.0011],
        [ 0.0416, -0.0197,  0.0055,  ...,  0.0388, -0.0344,  0.0106]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_sta